## Generate large frames for testing

In [11]:
import nbloader
from Frame2D_v03 import *

In [12]:
## NOTE: all units are kN and m

FD = {'storey_heights': [6.5] + [5.5]*20 + [7.0],   # m
      'bay_widths': [10.5,10,10,10,10,10.5],        # m
      'frame_spacing':8,                            # m, used only for load calculation
      'specified_loads':{'live':2.4,'dead':4.0,'snow':2.5,'wind':2.5},  # kPa
      'braced_bays':[0,2,3,5],
      'support_fixity': ['fx,fy,mz']*3 + ['fx,fy']*2 + ['fx,fy,mz']*2,
      'beam_size': 'W1000x222',
      'column_size': 'W360x216',
      'load_combination': '1.25*D + 1.4*W + 0.5*L',
      }

SaveData = True
ShowResults = True

In [13]:
FD

{'bay_widths': [10.5, 10, 10, 10, 10, 10.5],
 'beam_size': 'W1000x222',
 'braced_bays': [0, 2, 3, 5],
 'column_size': 'W360x216',
 'frame_spacing': 8,
 'load_combination': '1.25*D + 1.4*W + 0.5*L',
 'specified_loads': {'dead': 4.0, 'live': 2.4, 'snow': 2.5, 'wind': 2.5},
 'storey_heights': [6.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  7.0],
 'support_fixity': ['fx,fy,mz',
  'fx,fy,mz',
  'fx,fy,mz',
  'fx,fy',
  'fx,fy',
  'fx,fy,mz',
  'fx,fy,mz']}

In [14]:
def genframe(fd):
    h = fd['storey_heights']
    w = fd['bay_widths']
    s = fd['frame_spacing']
    nnodes = (len(h)+1)*(len(w)+1)
    
    # names of column stacks and floor levels
    bayline = [chr(ord('A')+i) for i in range(len(w)+1)]
    floorlev = [str(i) for i in range(len(h)+1)]
    
    # generate the nodes
    nodelist = []
    nidgrid = np.ndarray((len(h)+1,len(w)+1),dtype=np.object)
    for i in range(len(h)+1):
        y = sum(h[:i])*1000.
        for j in range(len(w)+1):
            x = sum(w[:j])*1000.
            nid = bayline[j]+floorlev[i]
            nodelist.append((nid,x,y))
            nidgrid[i,j] = nid
    nodes = pd.DataFrame(nodelist,columns=['ID','X','Y'])
    
    # generate the supports
    assert len(fd['support_fixity'])==nidgrid.shape[1]
    supplist = []
    for j,s in enumerate(fd['support_fixity']):
        nid = nidgrid[0,j]
        fix = s.strip().upper().split(',')
        if len(fix) < 3:
            fix += [np.nan] * (3-len(fix))
        supplist.append([nid,]+fix)
    supports = pd.DataFrame(supplist,columns=['ID','C0','C1','C2'])
    
    # generate columns
    columns = []
    for i in range(nidgrid.shape[0]-1):
        for j in range(nidgrid.shape[1]):
            nidj = nidgrid[i,j]
            nidk = nidgrid[i+1,j]
            mid = 'C' + nidj + nidk
            columns.append((mid,nidj,nidk))
    
    # generate beams
    beams = []
    roofbeams = []
    pinnedbeams = []
    for i in range(1,nidgrid.shape[0]):
        beamlist = beams if i < nidgrid.shape[0]-1 else roofbeams
        for j in range(nidgrid.shape[1]-1):
            nidj = nidgrid[i,j]
            nidk = nidgrid[i,j+1]
            mid = 'B' + nidj + nidk
            beamlist.append((mid,nidj,nidk))
            if j not in fd['braced_bays']:
                pinnedbeams.append(mid)
    members = pd.DataFrame(columns+beams+roofbeams,columns=['ID','NODEJ','NODEK'])
    
    # generate releases
    rellist = []
    for mid in pinnedbeams:
        rellist.append((mid,'MZJ'))
        rellist.append((mid,'MZK'))
    releases = pd.DataFrame(rellist,columns=['ID','R'])
    
    # generate properties
    proplist = []
    size = fd['column_size']
    for mid,j,k in columns:
        proplist.append((mid,size,np.nan,np.nan))
        size = np.nan
    size = fd['beam_size']
    for mid,j,k in beams+roofbeams:
        proplist.append((mid,size,np.nan,np.nan))
        size = np.nan
    properties = pd.DataFrame(proplist,columns=['ID','SIZE','Ix','A'])
    
    # generate node loads (wind from left)
    nloadlist = []
    SL = fd['specified_loads']   # area loads
    L = {'S':0., 'W':0., 'D':0., 'L':0., 'E':0.}
    for i in range(1,nidgrid.shape[0]+1):
        H = (sum(h[:i+1])-sum(h[:i-1]))/2.
        L['W'] = H*fd['frame_spacing']*SL['wind']
        FL = eval(fd['load_combination'],L)
        if FL != 0.:
            nloadlist.append((nidgrid[i,0],'FX',FL*1000.))
    node_loads = pd.DataFrame(nloadlist,columns=['ID','DIRN','F'])
    
    # generate member loads
    L = {'S':0., 'W':0., 'D':0., 'L':0., 'E':0.}
    L['D'] = SL['dead']
    L['L'] = SL['live']
    F = -eval(fd['load_combination'],L)
    UDL = F*fd['frame_spacing']
    mloadlist = [(mid,'UDL',UDL,np.nan,np.nan,np.nan,np.nan) for mid,nj,nk in beams]
    L['L'] = 0.
    L['S'] = SL['snow']
    F = -eval(fd['load_combination'],L)
    UDL = F*fd['frame_spacing']
    mloadlist += [(mid,'UDL',UDL,np.nan,np.nan,np.nan,np.nan) for mid,nj,nk in roofbeams]
    member_loads = pd.DataFrame(mloadlist,columns='ID,TYPE,W1,W2,A,B,C'.split(','))
    
    frame = Frame2D()
    frame.install_nodes(nodes)
    frame.install_supports(supports)
    frame.install_members(members)
    frame.install_releases(releases)
    frame.install_properties(properties)
    frame.install_node_loads(node_loads)
    frame.install_member_loads(member_loads)
    
    return frame

In [15]:
##%%timeit

f = genframe(FD)
## roof: 10 loops, best of 3: 32.8 ms per loop / ndof = 120, nfree = 106, nconstraints = 14

In [16]:
%matplotlib notebook
f.plot()

<IPython.core.display.Javascript object>

In [17]:
f.doall(show=ShowResults)


Time to solve = 0.06 sec.

ndof = 483, nfree = 464, nconstraints = 19



Nodes:

Node          X         Y  Constraints  DOF #s
----      -----     -----  -----------  ------
A0            0         0  FX,FY,MZ     464,465,466
B0        10500         0  FX,FY,MZ     467,468,469
C0        20500         0  FX,FY,MZ     470,471,472
D0        30500         0  FX,FY        473,474,0
E0        40500         0  FX,FY        475,476,1
F0        50500         0  FX,FY,MZ     477,478,479
G0        61000         0  FX,FY,MZ     480,481,482
A1            0      6500               2,3,4
B1        10500      6500               5,6,7
C1        20500      6500               8,9,10
D1        30500      6500               11,12,13
E1        40500      6500               14,15,16
F1        50500      6500               17,18,19
G1        61000      6500               20,21,22
A2            0     12000               23,24,25
B2        10500     12000               26,27,28
C2        20500     12000      

In [18]:
if SaveData:
    NS = len(FD['storey_heights'])
    NB = len(FD['bay_widths'])
    f.saveStructuralData('l{}x{}'.format(NS,NB),mkdir=True,save_description=True)

In [19]:
f.doall(pdelta=True,maxiter=40,show=ShowResults)

iter=1, max D=1789.55136696, max chg=363.108460295, max % chg=20.2904743053
iter=2, max D=1886.48257972, max chg=96.9313106635, max % chg=5.13820332642
iter=3, max D=1913.24446019, max chg=26.7618972063, max % chg=1.39877040092
iter=4, max D=1920.85518309, max chg=7.61072456253, max % chg=0.396215426833
iter=5, max D=1923.08059088, max chg=2.22540779434, max % chg=0.115720984596
iter=6, max D=1923.74863026, max chg=0.668039378337, max % chg=0.0347259183362
iter=7, max D=1923.95410688, max chg=0.20547662298, max % chg=0.0106799129067
iter=8, max D=1924.01870535, max chg=0.0645984677942, max % chg=0.00335747607934

Time to solve = 0.26 sec.

ndof = 483, nfree = 464, nconstraints = 19



Nodes:

Node          X         Y  Constraints  DOF #s
----      -----     -----  -----------  ------
A0            0         0  FX,FY,MZ     464,465,466
B0        10500         0  FX,FY,MZ     467,468,469
C0        20500         0  FX,FY,MZ     470,471,472
D0        30500         0  FX,FY        473,474,

In [20]:
if SaveData:
    f.saveStructuralData('l{}x{}pd'.format(NS,NB),mkdir=True,save_description=True)